# OPTaaS Tutorial

OPTaaS is an optimization service developed by Mind Foundry. It can be used to optimize any function you wish to use; all it needs to know is the parameter space over which the function operates.

The procedure is simple: you create a task, which is the definition of the parameters of the function, and send it to OPTaaS. The service will then suggest a configuration (set of values for your parameters) to evaluate. You evaluate the configuration locally and send the score you obtained back to OPTaaS. You then repeat the process until you're happy that the score is good, or that there hasn't been enough progress in a while. 

Here we demonstrate how to use OPTaaS to optimize a scikit-learn classification pipeline.

## Load your dataset

We will run a classification pipeline using the German Credit Data available [here](https://newonlinecourses.science.psu.edu/stat857/node/215/). The data contains 1000 rows, with 20 feature columns and 1 target column which includes 2 classes.

In [1]:
import pandas as pd

data = pd.read_csv('german_credit.csv')
features = data[data.columns.drop(['Creditability'])]
target = data['Creditability']

## Create your pipeline

We can then assemble a simple pipeline: we reduce the input dimensions with PCA, and then discover the type of flower with a Random Forest Classifier.

We create a function that will set the parameters of our pipeline based on the values that OPTaaS will generate.

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

def make_pipeline(**params):
    pipeline = Pipeline([('pca', PCA()), ('rf', RandomForestClassifier())])
    pipeline.set_params(**params)
    return pipeline

make_pipeline()

Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
          ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

## Define the parameters to be optimized

Let's now define the parameters for our task. A list of available parameter types can be found [here](autogen/mindfoundry.optaas.client.html#module-mindfoundry.optaas.client.parameter).

We can use *constraints* to make sure that the parameter values are always valid, e.g. that `max_features` in our classifier will never be larger than `n_components` in PCA. (see also: [Constraints](constraints.ipynb))

scikit-learn pipelines follow a naming convention, so we abide to that convention here. This allows us to plug them in directly into `Pipeline.set_params()`.

Note that you don't have to define every single parameter of your pipelines, just the ones you would like to optimize. If you don't know what parameters will lead to better results, give them all to OPTaaS and let it do the hard work for you. However, the more parameters you have, the harder it is to optimize over them, so you will need more iterations to achieve comparable results. 

In [3]:
from mindfoundry.optaas.client.parameter import IntParameter, BoolParameter, CategoricalParameter, Distribution
from mindfoundry.optaas.client.constraint import Constraint

feature_count = len(features.columns)
n_components = IntParameter('pca__n_components', minimum=1, maximum=feature_count)
max_features = IntParameter('rf__max_features', minimum=1, maximum=feature_count, optional=True, include_in_default=False)
constraint = Constraint(when=max_features.is_present(), then=max_features < n_components) 

parameters = [
    # PCA parameters
    n_components,
    BoolParameter('pca__whiten'),
    
    # Random Forest parameters
    max_features,
    IntParameter('rf__n_estimators', minimum=10, maximum=1000, default=10, distribution=Distribution.LOGUNIFORM),
    CategoricalParameter('rf__criterion', values=['entropy', 'gini']),
    IntParameter('rf__max_depth', minimum=1, maximum=100, optional=True),
    IntParameter('rf__min_samples_split', minimum=2, maximum=20),
    IntParameter('rf__min_samples_leaf', minimum=1, maximum=20),
]

display(parameters)

[{'id': '2592603269720', 'name': 'pca__n_components', 'type': 'integer', 'minimum': 1, 'maximum': 20},
 {'id': '2592603252832', 'name': 'pca__whiten', 'type': 'boolean'},
 {'id': '2592603269888', 'name': 'rf__max_features', 'type': 'integer', 'optional': True, 'includeInDefault': False, 'minimum': 1, 'maximum': 20},
 {'id': '2592603252888', 'name': 'rf__n_estimators', 'type': 'integer', 'default': 10, 'minimum': 10, 'maximum': 1000, 'distribution': 'LogUniform'},
 {'id': '2592603252944', 'name': 'rf__criterion', 'type': 'categorical', 'enum': ['entropy', 'gini']},
 {'id': '2592603253000', 'name': 'rf__max_depth', 'type': 'integer', 'optional': True, 'minimum': 1, 'maximum': 100},
 {'id': '2592603253056', 'name': 'rf__min_samples_split', 'type': 'integer', 'minimum': 2, 'maximum': 20},
 {'id': '2592603253112', 'name': 'rf__min_samples_leaf', 'type': 'integer', 'minimum': 1, 'maximum': 20}]

## Connect to the OPTaaS server using your API Key

We now create a client, and connect to the web service that will perform our optimization. You will need to input your personal API key. Make sure you keep your key private and don't commit it to your version control system. 

In [4]:
from mindfoundry.optaas.client.client import OPTaaSClient

client = OPTaaSClient('<URL of your OPTaaS server>', '<Your OPTaaS API key>')

## Create your Task

In [5]:
task = client.create_task(
    title='My Pipeline Optimization', 
    parameters=parameters,
    constraints=[constraint]
)

## Generate the first configuration

Here OPTaaS gives us the first configuration (set of parameter values) for our problem. 

In [6]:
configuration = task.generate_configuration()

display(configuration)

{ 'id': 'a485cfc1-0140-4e34-bb46-32b026a5ab8b',
  'type': 'default',
  'values': { 'pca__n_components': 10,
              'pca__whiten': False,
              'rf__criterion': 'entropy',
              'rf__max_depth': 50,
              'rf__min_samples_leaf': 10,
              'rf__min_samples_split': 11,
              'rf__n_estimators': 10}}

## Use the configuration to set pipeline parameters and calculate the result

We can plug this configuration into our pipeline, run it and score it. We wrapped this into a function so that we can call it multiple times.

In [7]:
from sklearn.model_selection import cross_val_score

def get_result(configuration):
    pipeline = make_pipeline()
    scores = cross_val_score(pipeline, features, target, scoring='f1_micro')
    mean_score = scores.mean()
    return list(scores), mean_score

scores, mean_score = get_result(configuration)

display(f'Mean score: {mean_score:.3f}')

'Mean score: 0.692'

## Record the result in OPTaaS and generate the next configuration to try

The score we obtained must now be passed back to OPTaaS, which will use it to generate and return a new configuration.

You can store additional user-defined data along with your result, but that's entirely optional and for your convenience only. For example, you can store the individual scores for each category so you can easily reference them later.

In [8]:
configuration = task.record_result(configuration, score=mean_score, user_defined_data=scores)

## Repeat as necessary

Repeat the process until you're happy with the result or you've run out of time. How many times should we run it? There is no single answer to this question, but you can look for a certain amount of improvement at each iteration, and stop when you haven't improved in a while. Again this is up to you, and depending on how long you are prepared to wait and perhaps even on how much it costs to run each evaluation on your hardware, especially if you're using a cloud computing service. The more the better (remembering that harder problems, i.e. problems with more parameters, will need more iterations to find a decent solution).

In [9]:
number_of_iterations = 20

for _ in range(number_of_iterations):
    display("OPTaaS recommendation:", configuration)
    scores, mean_score = get_result(configuration)
    display(f'Mean score: {mean_score:.3f}')
    configuration = task.record_result(configuration, score=mean_score, user_defined_data=scores)

'OPTaaS recommendation:'

{ 'id': '7b00c6ea-3237-43a5-9429-227c24837485',
  'type': 'exploration',
  'values': { 'pca__n_components': 1,
              'pca__whiten': True,
              'rf__criterion': 'entropy',
              'rf__min_samples_leaf': 1,
              'rf__min_samples_split': 7,
              'rf__n_estimators': 12}}

'Mean score: 0.682'

'OPTaaS recommendation:'

{ 'id': '887f4904-96a0-44e4-8c37-3043f5007f4d',
  'type': 'exploration',
  'values': { 'pca__n_components': 19,
              'pca__whiten': True,
              'rf__criterion': 'gini',
              'rf__min_samples_leaf': 20,
              'rf__min_samples_split': 2,
              'rf__n_estimators': 113}}

...

{ 'id': '6b7fb4c7-1127-449a-8621-a4840f1ce4a8',
  'type': 'exploration',
  'values': { 'pca__n_components': 11,
              'pca__whiten': True,
              'rf__criterion': 'entropy',
              'rf__max_depth': 69,
              'rf__min_samples_leaf': 1,
              'rf__min_samples_split': 2,
              'rf__n_estimators': 440}}

'Mean score: 0.707'

'OPTaaS recommendation:'

{ 'id': '16154785-688d-4cbc-8208-e9d7da97c048',
  'type': 'exploitation',
  'values': { 'pca__n_components': 17,
              'pca__whiten': False,
              'rf__criterion': 'entropy',
              'rf__min_samples_leaf': 20,
              'rf__min_samples_split': 18,
              'rf__n_estimators': 22}}

'Mean score: 0.695'

## Complete your task

Tell OPTaaS that you're satisfied and you won't run any more experiments, then go ahead and get your final result out. Job done!

In [10]:
task.complete()

best_result, best_configuration = task.get_best_result_and_configuration()

display(best_result, best_configuration)

{ 'configuration': 'd8600a7a-3f5b-469b-a70b-0fca521ac48e',
  'id': 102,
  'score': 0.7309644974315632,
  'user_defined_data': [ 0.7664670658682635,
                         0.7087087087087086,
                         0.7177177177177178]}

{ 'id': 'd8600a7a-3f5b-469b-a70b-0fca521ac48e',
  'type': 'exploration',
  'values': { 'pca__n_components': 14,
              'pca__whiten': False,
              'rf__criterion': 'entropy',
              'rf__min_samples_leaf': 20,
              'rf__min_samples_split': 18,
              'rf__n_estimators': 109}}